# Regroupement des données par un pas de temps

In [ ]:
import pandas as pd
import numpy as np

#### Paramètres

In [ ]:
data_path = "data/data_processed.csv"
step_group = "90min"
out_path = "data/data_processed_{}_257".format(step_group)

#### Chargement et one hot CBN

In [ ]:
data = pd.read_csv(data_path)

data['date'] = data['date'].apply(lambda x : pd.Timestamp(x))

data['CBN'] = pd.Categorical(data['CBN'])

# Créer les one hot encodings pour chaque variabble
dfDummies_cbn = pd.get_dummies(data['CBN'], prefix = 'CBN')

# Concatener les one hot encodings avec les données
data = pd.concat([data, dfDummies_cbn], axis=1)

# Suppression de la variable CBBN
data.drop(columns=['CBN'], inplace=True)

# On ordonne les data par "date"
data.sort_values(by='date', inplace=True)

# Réindexage des lignes
data.reset_index(inplace=True)
data.drop(columns=['index'], inplace=True)

In [ ]:
data.head()

#### Groupement  des Data

In [ ]:
grouped_data = data.groupby(pd.Grouper(key="date", freq=step_group)).sum()

grouped_data.reset_index(inplace=True)

In [ ]:
grouped_data.head()

#### Ajout variables jour seamine, mois et heure

In [ ]:
# Ajout variables
grouped_data['month'] = grouped_data['date'].apply(lambda x : x.month)
grouped_data['weekday'] = grouped_data['date'].apply(lambda x : x.weekday())
grouped_data['hour'] = grouped_data['date'].apply(lambda x : x.hour)


grouped_data['month'] = pd.Categorical(grouped_data['month'])
grouped_data['weekday'] = pd.Categorical(grouped_data['weekday'])
grouped_data['hour'] = pd.Categorical(grouped_data['hour'])

# Créer les one hot encodings pour chaque variabble
dfDummies_month = pd.get_dummies(grouped_data['month'], prefix = 'month')
dfDummies_weekday = pd.get_dummies(grouped_data['weekday'], prefix = 'weekday')
dfDummies_hour = pd.get_dummies(grouped_data['hour'], prefix = 'hour')

# Concatener les one hot encodings avec les données
grouped_data = pd.concat([grouped_data, dfDummies_weekday, dfDummies_month, dfDummies_hour], axis=1)

# Suppression de la variable mois et jour de la semaine
grouped_data.drop(columns=['month', 'weekday', 'hour'], inplace=True)

In [ ]:
grouped_data.to_csv(out_path, index=False)

### Test

In [ ]:
test_df = pd.read_csv(out_path)

test_df.head()